In [1]:
# Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn import metrics
import statsmodels.api as sm

from sklearn.preprocessing import OneHotEncoder

from sklearn.neighbors import KNeighborsRegressor

from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import BaggingRegressor, VotingRegressor

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor

In [2]:
shift_df = pd.read_csv('data/shift_details.csv')

In [3]:
shift_df.head()

,SITE_NAME,DATE1,DAY_OF_WEEK,INSPECTOR_ID,PAY_VOL,SHIFT_START,SHIFT_END,TOTALINSP,NUMINVASIVE,TOWN,WATERBODY,SHIFT_LENGTH,DATE,month,year
0,Launch Drive,05/28/2021,Fri,4771,1,12:00,18:00,33.0,0.0,Launch Drive,Cobbosseecontee Lake,360.0,2021-05-28,5,2021
1,East Winthrop,05/28/2021,Fri,4485,1,12:00,18:00,2.0,0.0,Winthrop,Cobbosseecontee Lake,360.0,2021-05-28,5,2021
2,Augusta West Kampground,05/28/2021,Fri,4769,1,12:00,18:00,1.0,0.0,Winthrop,Annabessacook Lake,360.0,2021-05-28,5,2021
3,Whippoorwill Road,05/28/2021,Fri,4174,1,12:00,18:00,13.0,0.0,Litchfield,Woodbury Pond,360.0,2021-05-28,5,2021
4,Thorofare Rd,05/29/2021,Sat,4944,1,07:00,17:00,11.0,0.0,Litchfield,Pleasant Pond,600.0,2021-05-29,5,2021


In [4]:
y = shift_df['TOTALINSP']
X = shift_df.drop(columns=['DATE1', 'TOTALINSP', 'NUMINVASIVE', 'TOWN', 'WATERBODY', 'DATE', 'INSPECTOR_ID'])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=.3)

In [37]:
X_train

,SITE_NAME,DAY_OF_WEEK,PAY_VOL,SHIFT_START,SHIFT_END,SHIFT_LENGTH,month,year
1510,Old Kents Hill Road,Mon,1,09:00,19:00,600.0,5,2023
1061,Launch Drive Cobbosseecontee,Sun,1,13:00,19:00,360.0,7,2022
2040,East Winthrop Cobbosseecontee,Thu,1,13:00,19:00,360.0,8,2023
123,Whippoorwill Road,Sun,1,13:00,19:00,360.0,6,2021
729,Rt 41 North Basin Maranacook,Sun,1,13:00,19:00,360.0,6,2022
...,...,...,...,...,...,...,...,...
1638,East Winthrop Cobbosseecontee,Tue,1,13:00,19:00,360.0,6,2023
1095,Launch Drive Cobbosseecontee,Fri,1,07:00,13:00,360.0,7,2022
1130,Rt 41 North Basin Maranacook,Sun,1,07:00,13:00,360.0,7,2022
1294,Rt 41 North Basin Maranacook,Sun,1,13:00,19:00,360.0,8,2022


In [12]:
# Calculate the baseline r**2
shift_df['y_preds'] = np.mean(shift_df.TOTALINSP)

In [13]:
metrics.mean_squared_error(shift_df.TOTALINSP, shift_df.y_preds)

294.8412280701754

In [15]:
metrics.r2_score(shift_df.TOTALINSP, shift_df.y_preds)

0.0

In [ ]:
# The baseline r2 is very low- 0

In [6]:
categorical_columns = ['SITE_NAME', 'DAY_OF_WEEK', 'month', 'year']
oh = OneHotEncoder(handle_unknown='ignore', drop='first')
X_train_transformed = oh.fit_transform(X_train[categorical_columns])
X_test_transformed= oh.transform(X_test[categorical_columns])

In [43]:
# Basic Linear Regression Model
lr = LinearRegression()
lr.fit(X_train_transformed, y_train)

LinearRegression()

In [44]:
lr.score(X_train_transformed, y_train)

0.3142477893028155

In [46]:
lr.score(X_test_transformed, y_test)

0.26526984756509553

In [16]:
# Random forest model
rf = RandomForestRegressor(n_estimators=100, n_jobs=2, random_state=42, oob_score=True)
rf.fit(X_train_transformed, y_train)
print(rf.score(X_train_transformed, y_train))
print(rf.score(X_test_transformed, y_test))

0.5823881813796656
0.28061943955375523


In [17]:
# Extra Trees model
et = ExtraTreesRegressor(n_estimators=100, n_jobs=2, random_state=42)
et.fit(X_train_transformed, y_train)
print(et.score(X_train_transformed, y_train))
print(et.score(X_test_transformed, y_test))

0.5982039094617482
0.19747301885075785


Both tree models are very ovefit- adding regularization and gridsearching over to find best params could improve the models and improve the testing r2. However, despite being overfit, the random forest model with an r2 of .28 is promising. Both models are potentially decent starting points and using a combination of new data (weather) and feature engineering (holidays and inspector experience) could further improve the models' success. 

In [20]:
# AdaBoost Model
ada = AdaBoostRegressor()
ada.fit(X_train_transformed, y_train)
print(ada.score(X_train_transformed, y_train))
print(ada.score(X_test_transformed, y_test))

0.1631178499776783
0.057888955598118574


In [22]:
# Gradient Boost Model
grad = GradientBoostingRegressor()
grad.fit(X_train_transformed, y_train)
print(grad.score(X_train_transformed, y_train))
print(grad.score(X_test_transformed, y_test))

0.4110300421256725
0.2711720175379255


In [27]:
# KNN model
sc = StandardScaler()
sc.fit(X_train_transformed)
X_train_sc = sc.tranform(X_train_transformed)
X_test_sc = sc.transform(X_test_transformed)

knn = KNeighborsRegressor()
knn.fit(X_train_sc, y_train)
print(knn.score(X_train_sc, y_train))
print(knn.score(X_test_sc, y_test))

ValueError: Cannot center sparse matrices: pass `with_mean=False` instead. See docstring for motivation and alternatives.

In [43]:
# Initial model with weather

df = pd.read_csv('data/df_with_weather.csv')

In [44]:
df.head()

,DATE1,DAY_OF_WEEK,SITE_NAME,TOWN,WATERBODY,INSPECTOR_ID,PAY_VOL,SHIFT_START,TRAILERS,SHIFT_END,SHIFT_LENGTH,TOTALINSP,PRCP,TMAX,TMIN,TOBS
0,2021-05-28,Fri,Launch Drive,Monmouth,Cobbosseecontee Lake,4771,Paid,12:00,10.0,18:00,360,33.0,0.00,72.0,45.0,49.0
1,2021-05-28,Fri,Whippoorwill Road,Litchfield,Woodbury Pond,4174,Paid,12:00,1.0,18:00,360,13.0,0.00,72.0,45.0,49.0
2,2021-05-28,Fri,Augusta West Kampground,Winthrop,Annabessacook Lake,4769,Paid,12:00,0.0,18:00,360,1.0,0.00,72.0,45.0,49.0
3,2021-05-28,Fri,East Winthrop,Winthrop,Cobbosseecontee Lake,4485,Paid,12:00,1.0,18:00,360,2.0,0.00,72.0,45.0,49.0
4,2021-05-29,Sat,Thorofare Rd,Litchfield,Pleasant Pond,4944,Paid,7:00,1.0,17:00,600,11.0,0.02,62.0,45.0,46.0


In [45]:
df.drop(columns=['TRAILERS'], inplace=True)

In [46]:
df.isna().sum()

DATE1            0
DAY_OF_WEEK      0
SITE_NAME        0
TOWN             0
WATERBODY        0
INSPECTOR_ID     0
PAY_VOL          0
SHIFT_START      0
SHIFT_END        0
SHIFT_LENGTH     0
TOTALINSP        2
PRCP            27
TMAX            91
TMIN            95
TOBS            55
dtype: int64

In [47]:
df.shape

(2669, 15)

In [48]:
df.dropna(inplace=True)

In [49]:
df.isna().sum()

DATE1           0
DAY_OF_WEEK     0
SITE_NAME       0
TOWN            0
WATERBODY       0
INSPECTOR_ID    0
PAY_VOL         0
SHIFT_START     0
SHIFT_END       0
SHIFT_LENGTH    0
TOTALINSP       0
PRCP            0
TMAX            0
TMIN            0
TOBS            0
dtype: int64

In [50]:
df.shape

(2565, 15)

In [51]:
df.head()

,DATE1,DAY_OF_WEEK,SITE_NAME,TOWN,WATERBODY,INSPECTOR_ID,PAY_VOL,SHIFT_START,SHIFT_END,SHIFT_LENGTH,TOTALINSP,PRCP,TMAX,TMIN,TOBS
0,2021-05-28,Fri,Launch Drive,Monmouth,Cobbosseecontee Lake,4771,Paid,12:00,18:00,360,33.0,0.00,72.0,45.0,49.0
1,2021-05-28,Fri,Whippoorwill Road,Litchfield,Woodbury Pond,4174,Paid,12:00,18:00,360,13.0,0.00,72.0,45.0,49.0
2,2021-05-28,Fri,Augusta West Kampground,Winthrop,Annabessacook Lake,4769,Paid,12:00,18:00,360,1.0,0.00,72.0,45.0,49.0
3,2021-05-28,Fri,East Winthrop,Winthrop,Cobbosseecontee Lake,4485,Paid,12:00,18:00,360,2.0,0.00,72.0,45.0,49.0
4,2021-05-29,Sat,Thorofare Rd,Litchfield,Pleasant Pond,4944,Paid,7:00,17:00,600,11.0,0.02,62.0,45.0,46.0


In [52]:
# Convert 'date1' to datetime
df['DATE'] = pd.to_datetime(df['DATE1'], format='%Y/%m/%d')

# Create 'month' and 'year' columns
df['month'] = df['DATE'].dt.month
df['year'] = df['DATE'].dt.year

In [53]:
df.head()

,DATE1,DAY_OF_WEEK,SITE_NAME,TOWN,WATERBODY,INSPECTOR_ID,PAY_VOL,SHIFT_START,SHIFT_END,SHIFT_LENGTH,TOTALINSP,PRCP,TMAX,TMIN,TOBS,DATE,month,year
0,2021-05-28,Fri,Launch Drive,Monmouth,Cobbosseecontee Lake,4771,Paid,12:00,18:00,360,33.0,0.00,72.0,45.0,49.0,2021-05-28,5,2021
1,2021-05-28,Fri,Whippoorwill Road,Litchfield,Woodbury Pond,4174,Paid,12:00,18:00,360,13.0,0.00,72.0,45.0,49.0,2021-05-28,5,2021
2,2021-05-28,Fri,Augusta West Kampground,Winthrop,Annabessacook Lake,4769,Paid,12:00,18:00,360,1.0,0.00,72.0,45.0,49.0,2021-05-28,5,2021
3,2021-05-28,Fri,East Winthrop,Winthrop,Cobbosseecontee Lake,4485,Paid,12:00,18:00,360,2.0,0.00,72.0,45.0,49.0,2021-05-28,5,2021
4,2021-05-29,Sat,Thorofare Rd,Litchfield,Pleasant Pond,4944,Paid,7:00,17:00,600,11.0,0.02,62.0,45.0,46.0,2021-05-29,5,2021


In [54]:
df.SHIFT_START = pd.to_datetime(df.SHIFT_START, format='%H:%M')

In [14]:
#df.SHIFT_END = pd.to_datetime(df.SHIFT_END)

In [15]:
X = df.drop(columns=['DATE1', 'TOWN', 'WATERBODY', 'INSPECTOR_ID', 'TOTALINSP', 'TOBS', 'DATE'])
y = df.TOTALINSP            

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=.2)

In [17]:
categorical_columns = ['SITE_NAME', 'DAY_OF_WEEK', 'month', 'year', 'PAY_VOL']
oh = OneHotEncoder(handle_unknown='ignore', drop='first')
X_train_transformed = oh.fit_transform(X_train[categorical_columns])
X_test_transformed= oh.transform(X_test[categorical_columns])

In [18]:
lr = LinearRegression()
lr.fit(X_train_transformed, y_train)
print(lr.score(X_train_transformed, y_train))
print(lr.score(X_test_transformed, y_test))

0.3513358726642821
0.33841873049366566


In [19]:
rf = RandomForestRegressor()
rf.fit(X_train_transformed, y_train)
print(rf.score(X_train_transformed, y_train))
print(rf.score(X_test_transformed, y_test))      

0.6438182757723909
0.3199733991337518


In [20]:
et = ExtraTreesRegressor()
et.fit(X_train_transformed, y_train)
print(et.score(X_train_transformed, y_train))
print(et.score(X_test_transformed, y_test))

0.6573212547527139
0.2538127489928671


From these initial models, adding weather data seems to improve the model performance significantly. The models become significantly less overgit. The next step will be to try to add in the holidays as another column, and then to use feature engineering with shift length, shift start and shift end to attempt to somehow capture time of day.

The models were all less overfit, however the tree models are still overfit. Parameter optimization could help reduce the overfit as well to balance out the metrics of the tree based models between the training and test scores. 

#### Feature Engineering Options:
- Transform the shift_start and end_shift columns from numbers to a categorical column of morning, late morning, afternoon, late afternoon, and evening
- Interaction variables?
- Holidays

In [21]:
# Converting times to 'time of day'
df.head(1)

,DATE1,DAY_OF_WEEK,SITE_NAME,TOWN,WATERBODY,INSPECTOR_ID,PAY_VOL,SHIFT_START,SHIFT_END,SHIFT_LENGTH,TOTALINSP,PRCP,TMAX,TMIN,TOBS,DATE,month,year
0,2021-05-28,Fri,Launch Drive,Monmouth,Cobbosseecontee Lake,4771,Paid,2023-12-03 12:00:00,2023-12-03 18:00:00,360,33.0,0.0,72.0,45.0,49.0,2021-05-28,5,2021


In [40]:
df.SHIFT_START.value_counts()

7:00     638
13:00    604
07:00    264
09:00    191
12:00    164
        ... 
5:25       1
6:10       1
6:15       1
6:47       1
09:40      1
Name: SHIFT_START, Length: 155, dtype: int64

In [41]:
df.dtypes

DATE1                   object
DAY_OF_WEEK             object
SITE_NAME               object
TOWN                    object
WATERBODY               object
INSPECTOR_ID             int64
PAY_VOL                 object
SHIFT_START             object
SHIFT_END               object
SHIFT_LENGTH             int64
TOTALINSP              float64
PRCP                   float64
TMAX                   float64
TMIN                   float64
TOBS                   float64
DATE            datetime64[ns]
month                    int64
year                     int64
dtype: object

In [55]:
df['SHIFT_START_CATEGORY'] = pd.cut(pd.to_datetime(df['SHIFT_START'], format='%H:%M').dt.hour,
                                    bins =[-1, 8, 11, 14, 17, 24],
                                    labels=['Early Morning', 'Morning', 'Afternoon', 'Late Afternoon', 'Evening'])
                        

In [56]:
df.isna().sum()

DATE1                   0
DAY_OF_WEEK             0
SITE_NAME               0
TOWN                    0
WATERBODY               0
INSPECTOR_ID            0
PAY_VOL                 0
SHIFT_START             0
SHIFT_END               0
SHIFT_LENGTH            0
TOTALINSP               0
PRCP                    0
TMAX                    0
TMIN                    0
TOBS                    0
DATE                    0
month                   0
year                    0
SHIFT_START_CATEGORY    0
dtype: int64

In [57]:
df.SHIFT_START_CATEGORY.value_counts()

Early Morning     1202
Afternoon          871
Morning            371
Late Afternoon     105
Evening             16
Name: SHIFT_START_CATEGORY, dtype: int64

In [61]:
df['SHIFT_END_CATEGORY'] = pd.cut(pd.to_datetime(df['SHIFT_END'], format='%H:%M').dt.hour,
                                    bins =[-1, 8, 11, 14, 17, 19, 24],
                                    labels=['Early Morning', 'Morning', 'Afternoon', 'Late Afternoon', 'Evening', 'Night'])
                        

In [63]:
df.SHIFT_END_CATEGORY.value_counts()

Evening           1004
Afternoon          810
Late Afternoon     511
Morning            108
Early Morning       88
Night               44
Name: SHIFT_END_CATEGORY, dtype: int64

In [65]:
X1 = df.drop(columns=['DATE1', 'TOWN', 'WATERBODY', 'INSPECTOR_ID', 'TOTALINSP', 'TOBS', 'DATE', 'SHIFT_START', 'SHIFT_END'])
y1 = df.TOTALINSP            

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, random_state=42, test_size=.2)

In [66]:
categorical_columns = ['SITE_NAME', 'DAY_OF_WEEK', 'month', 'year', 'PAY_VOL', 'SHIFT_START_CATEGORY', 'SHIFT_END_CATEGORY']
oh = OneHotEncoder(handle_unknown='ignore', drop='first')
X1_train_transformed = oh.fit_transform(X1_train[categorical_columns])
X1_test_transformed= oh.transform(X1_test[categorical_columns])

In [67]:
lr1 = LinearRegression()
lr1.fit(X1_train_transformed, y1_train)
print(lr1.score(X1_train_transformed, y1_train))
print(lr1.score(X1_test_transformed, y1_test))

0.43299897430845313
0.45589238044822944


In [68]:
rf1 = RandomForestRegressor()
rf1.fit(X1_train_transformed, y1_train)
print(rf1.score(X1_train_transformed, y1_train))
print(rf1.score(X1_test_transformed, y1_test))     

0.7976454375369557
0.38356603863290895


In [69]:
et1 =ExtraTreesRegressor()
et1.fit(X1_train_transformed, y1_train)
print(et1.score(X1_train_transformed, y1_train))
print(et1.score(X1_test_transformed, y1_test))      

0.8333177482959641
0.18941843140667491


In [70]:
# List of holidays
# 2021: Monday, May 31 Memorial Day, Sunday July 4 (Observed Monday), Monday September 6 (Labor Day), Sunday May 9 (Mother's Day), Father's Day (Sunday, June 20)
# 2022: Sunday May 08, Mother's Day, 'Monday May 30 Memorial Day, Monday September 5 Labor Day, Monday July 4, Sunday June 19 Father's Day
# 2023: Monday, May 29 Memorial Day, Monday June19 Juneteenth, Tuesday, July 04, Monday September 04 Labor Day, Mother's Day Sunday May 14, Father's Day Sunday June 18
#'2021-05-09', '2022-05-30', 
holiday_list = ['2021-05-31', '2021-07-04', '2021-09-06', '2021-06-20',
                '2022-05-08', '2022-05-30', '2022-07-04', '2022-06-19',
                '2023-05-29', '2023-06-19', '2023-07-04', '2023-09-04', '2023-06-18']


In [72]:
df['holiday']= np.where(df['DATE'].isin(holiday_list), 1, 0)

In [73]:
df.sample(25)

,DATE1,DAY_OF_WEEK,SITE_NAME,TOWN,WATERBODY,INSPECTOR_ID,PAY_VOL,SHIFT_START,SHIFT_END,SHIFT_LENGTH,...,PRCP,TMAX,TMIN,TOBS,DATE,month,year,SHIFT_START_CATEGORY,SHIFT_END_CATEGORY,holiday
624,2021-09-06,Mon,Launch Drive,Monmouth,Cobbosseecontee Lake,3796,Paid,1900-01-01 07:00:00,16:30,570,...,0.06,72.0,50.0,67.0,2021-09-06,9,2021,Early Morning,Late Afternoon,1
2319,2023-08-07,Mon,East Winthrop Cobbosseecontee,Winthrop,Cobbosseecontee Lake,5987,Paid,1900-01-01 13:00:00,18:00,300,...,0.00,82.0,60.0,68.0,2023-08-07,8,2023,Afternoon,Evening,0
2360,2023-08-12,Sat,East Winthrop Cobbosseecontee,Winthrop,Cobbosseecontee Lake,5670,Paid,1900-01-01 09:00:00,19:00,600,...,0.00,77.0,59.0,68.0,2023-08-12,8,2023,Morning,Evening,0
1632,2022-09-06,Tue,Lakeside Marina Cobbosseecontee,Winthrop,Cobbosseecontee Lake,5655,Paid,1900-01-01 14:00:00,17:00,180,...,1.07,60.0,58.0,60.0,2022-09-06,9,2022,Afternoon,Late Afternoon,0
870,2022-07-01,Fri,Whippoorwill Road,Litchfield,Woodbury Pond,3504,Paid,1900-01-01 12:00:00,18:00,360,...,0.00,79.0,57.0,70.0,2022-07-01,7,2022,Afternoon,Evening,0
1529,2022-08-28,Sun,Norcross Point South Basin Maranacook,Winthrop,Maranacook Lake,5644,Paid,1900-01-01 07:00:00,17:00,600,...,0.00,78.0,62.0,67.0,2022-08-28,8,2022,Early Morning,Late Afternoon,0
3,2021-05-28,Fri,East Winthrop,Winthrop,Cobbosseecontee Lake,4485,Paid,1900-01-01 12:00:00,18:00,360,...,0.00,72.0,45.0,49.0,2021-05-28,5,2021,Afternoon,Evening,0
1625,2022-09-05,Mon,Launch Drive Cobbosseecontee,Monmouth,Cobbosseecontee Lake,5637,Volunteer,1900-01-01 04:56:00,7:06,130,...,0.33,85.0,58.0,59.0,2022-09-05,9,2022,Early Morning,Early Morning,1
10,2021-05-29,Sat,Wilson Pond Road,Monmouth,Wilson Pond,4770,Paid,1900-01-01 07:00:00,17:00,600,...,0.02,62.0,45.0,46.0,2021-05-29,5,2021,Early Morning,Late Afternoon,0
1647,2022-09-10,Sat,East Winthrop Cobbosseecontee,Winthrop,Cobbosseecontee Lake,5648,Paid,1900-01-01 07:00:00,13:00,360,...,0.00,83.0,58.0,64.0,2022-09-10,9,2022,Early Morning,Afternoon,0


In [74]:
X2 = df.drop(columns=['DATE1', 'TOWN', 'WATERBODY', 'INSPECTOR_ID', 'TOTALINSP', 'TOBS', 'DATE', 'SHIFT_START', 'SHIFT_END'])
y2 = df.TOTALINSP            

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, random_state=42, test_size=.2)

In [75]:
categorical_columns = ['SITE_NAME', 'DAY_OF_WEEK', 'month', 'year', 'PAY_VOL', 'SHIFT_START_CATEGORY', 'SHIFT_END_CATEGORY']
oh = OneHotEncoder(handle_unknown='ignore', drop='first')
X2_train_transformed = oh.fit_transform(X2_train[categorical_columns])
X2_test_transformed= oh.transform(X2_test[categorical_columns])

In [76]:
lr2 = LinearRegression()
lr2.fit(X2_train_transformed, y2_train)
print(lr2.score(X2_train_transformed, y2_train))
print(lr2.score(X2_test_transformed, y2_test))

0.43299897430845313
0.45589238044822944


In [77]:
rf2 = RandomForestRegressor()
rf2.fit(X2_train_transformed, y2_train)
print(rf2.score(X2_train_transformed, y2_train))
print(rf1.score(X2_test_transformed, y2_test))    

0.800217845750667
0.38356603863290895


In [79]:
et2 =ExtraTreesRegressor()
et2.fit(X2_train_transformed, y2_train)
print(et2.score(X2_train_transformed, y2_train))
print(et2.score(X2_test_transformed, y2_test))     

0.8333177482959641
0.18143892545551887


In [ ]:
holiday_list_weekends = ['2021-05-31', '2021-05-09', '2021-07-04', '2021-09-06', '2021-06-20',
                '2022-05-08', '2022-05-30', '2022-09-05', '2022-07-04', '2022-06-19',
                '2023-05-29', '2023-06-19', '2023-07-04', '2023-09-04', '2023-06-18']
